# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 24 нояб

__Жесткий дедлайн: 27 нояб


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов, 2 балл за доп вопросы, если возникнут, если вопросов не возникло, считаем, что 2 балла вы получили 

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd
from tqdm import tqdm

emails = pd.read_csv('emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [2]:
import re
import random

def clean_email(content):
    # Если строка начинается с "To:", удаляем её и две предыдущие строки
    lines = content.split('\n')
    stop_list = []
    for i in range(len(lines)):  
        # Если строка начинается с "To:", удаляем её и две предыдущие строки
        if lines[-i-1].strip().startswith("To:"):
            stop_list.append(len(lines)-i-1)
            stop_list.append(len(lines)-i-2)
            stop_list.append(len(lines)-i-3)
    stop_list= sorted(list(set(stop_list)))
    clean_lines = [elem for i, elem in enumerate(lines) if i not in stop_list]

    # Собираем текст обратно в одну строку
    content = "\n".join(clean_lines).strip()
    
    # Удаляем строки с метаданными (Message-ID, Date, From, To, Subject и другие)
    content = re.sub(r"(?i)(Message-ID|Date|From|Subject|Mime-Version|Content-Type|cc|Content-Transfer-Encoding|X-From|X-To|X-cc|X-bcc|X-Folder|X-Origin|X-FileName):.*", "", content)
    content = re.sub(r"--", "", content)
    content = re.sub(r"==", "", content)
    content = re.sub(r"__", "", content)
    content = re.sub(r"\*{2}", "", content)
    content = re.sub(r"IMAGE", "", content)
    
    # Убираем строку с «Forwarded by» и подобные повторяющиеся сообщения
    content = re.sub(r"(?i)^(.*?Forwarded by.*?)$", "", content, flags=re.MULTILINE)

    # Удаляем строки с символом '@'
    content = re.sub(r"^.*@.*$", "", content, flags=re.MULTILINE)

    # Убираем ненужные пустые строки (состоящие только из пробелов)
    content = re.sub(r"^\s*$", "", content, flags=re.MULTILINE)

    # Убираем все переносы строк, заменяя их на пробелы
    content = re.sub(r"\n", " ", content)
    content = re.sub(r'\s+', ' ', content)

    if len(content) == 0:
        return None

    if content[0] == ' ':
        content = content[1:]

    return content

email_text = emails['message'][random.randint(0, 517401)]
cleaned_email = clean_email(email_text)
print(cleaned_email)

tokens = re.findall(r'\w+|[^\w\s]', cleaned_email, re.UNICODE)
print(tokens)

Tracy, is there some particular reason I'm getting this notice? Debra Perlingiere is the paralegal that does the Master Firm Purchase/Sale Agreements.
['Tracy', ',', 'is', 'there', 'some', 'particular', 'reason', 'I', "'", 'm', 'getting', 'this', 'notice', '?', 'Debra', 'Perlingiere', 'is', 'the', 'paralegal', 'that', 'does', 'the', 'Master', 'Firm', 'Purchase', '/', 'Sale', 'Agreements', '.']


Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [3]:
from typing import List, defaultdict

class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        for word in tqdm(vocabulary):
            current_node = self.root
            for letter in word:
                if letter not in current_node.children:
                    current_node.children[letter] = PrefixTreeNode()
                current_node = current_node.children[letter]
            current_node.is_end_of_word = True

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        current_node = self.root
        for letter in prefix:
            if letter not in current_node.children:
                return []  # Если префикс не найден
            current_node = current_node.children[letter]
            
        result = []
        self._find_words_with_prefix(current_node, prefix, result)
        return result

    def _find_words_with_prefix(self, node: PrefixTreeNode, prefix: str, result: List[str]):
        """
        Рекурсивно находим все слова от текущей вершины
        """
        if node.is_end_of_word:
            result.append(prefix)
        
        # Проходим по всем дочерним вершинам
        for letter, child_node in node.children.items():
            self._find_words_with_prefix(child_node, prefix + letter, result)

In [4]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

100%|████████████████████████████████████████████████████| 6/6 [00:00<?, ?it/s]


Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [5]:
from typing import defaultdict

class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        self.volume, self.word_counts = self._get_word_counts(corpus) # Определяем общий объём слов и количество каждого уникального слова
        self.prefix_tree = PrefixTree(list(self.word_counts.keys()))

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words = self.prefix_tree.search_prefix(prefix) # Слова, начинающиеся на префикс
        probs = [self.word_counts[i]/self.volume for i in words] # Вероятность = количество слова / общее количесвто слов
        
        return words, probs

    def _get_word_counts(self, corpus: List[List[str]]) -> dict:
        word_counts = defaultdict(int)
        volume = 0
        # Подсчитываем частоту появления слов в списках
        for doc in tqdm(corpus):
            seen_words = set()  # Чтобы слово не учитывалось несколько раз в одном документе
            volume += len(doc)
            for word in doc:
                if word not in seen_words:
                    word_counts[word] += 1
                    seen_words.add(word)
        return volume, word_counts

In [6]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

100%|████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]


## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [7]:
class NGramLanguageModel:
    def __init__(self, corpus, n):
        self.n = n
        self.corpus = corpus
        
        # Словарь для хранения n-грамм: {префикс -> {следующее слово: частота}}
        self.ngram_counts = defaultdict(lambda: defaultdict(int))
        self.prefix_counts = defaultdict(int)
        
        # Строим модель на основе корпуса
        self._build_model()

    def _build_model(self):
        """
        Строит n-граммную модель на основе корпуса.
        """
        for sentence in tqdm(self.corpus):
            # Обрабатываем каждое предложение
            for i in range(len(sentence) - self.n):
                # Берем n-грамму из предложения
                prefix = tuple(sentence[i:i + self.n])  # (n-1)-грамма
                next_word = sentence[i + self.n]  # Следующее слово
                
                # Обновляем счетчик для префикса
                self.ngram_counts[prefix][next_word] += 1
                self.prefix_counts[prefix] += 1

    def get_next_words_and_probs(self, prefix: list) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после prefix,
        а также список вероятностей этих слов.
        """
        flag=0
        old_n = self.n
        if len(prefix) > self.n:
            prefix = prefix[-self.n:]
        elif len(prefix) < self.n:
            self.n = len(prefix)
            self._build_model()
            flag=1
            
        # Преобразуем prefix в кортеж
        prefix_tuple = tuple(prefix)
        
        # Получаем возможные следующие слова
        next_words = list(self.ngram_counts[prefix_tuple].keys())
        
        # Вычисляем вероятности
        probs = [
            count / self.prefix_counts[prefix_tuple]
            for count in self.ngram_counts[prefix_tuple].values()
        ]

        if flag:
            self.n = old_n
            self._build_model()
            
        return next_words, probs

In [8]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))
set(words_probs)

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

100%|████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [9]:
from copy import copy
import random

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model
        
    def inp2key(self, inp, l=10):
        k = {i:j for i,j in zip(*self.n_gram_model.get_next_words_and_probs(inp))}
        top_l = dict(sorted(k.items(), key=lambda item: item[1], reverse=True)[:l])
        return top_l
        
    def get_tree(self, inp, ns):
        f = self.inp2key(inp)
        s = {}
        for k in f.keys():
            s0 = self.inp2key(inp[1:]+[k])
            s0 = {tuple([k]+[key]): value*f[k] for key, value in s0.items()}
            s = s|s0
        for i in range(ns-2):
            f = s.copy()
            s = {}
            for k in f.keys():
                s0 = self.inp2key(tuple(inp[1:]+list(k)))
                s0 = {tuple(list(k)+[key]): value*f[k] for key, value in s0.items()}
                s = s|s0
        return s
        
    def suggest_text(self, text: list, n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """
        suggestions = []
        
        words, probs = self.word_completor.get_words_and_probs(text[-1])
        if len(probs) == 0:
            return [[]]
        else:
            first = words[probs.index(max(probs))]

        out = []
        inp = text[:-1]+[first]
        for i in range(n_words):
            words, probs = self.n_gram_model.get_next_words_and_probs(inp)
            if len(probs) == 0:
                return [[]]
            else:
                pred = words[probs.index(max(probs))]
                out.append(pred)
                inp.append(pred)

        most = inp[-n_words-1:]
        suggestions.append(list(most))
        if n_texts > 1:
            sp = list(self.get_tree(inp, n_words).keys())
            sp = [[first]+list(t) for t in sp]
            if most in sp:
                sp.remove(most)
            for t in random.sample(sp, min(len(sp), n_texts-1)):
                suggestions.append(t)
        
        return suggestions

In [10]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

100%|████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!

In [12]:
full_corpus = []
for i in tqdm(range(len(emails)):
    email_text = emails['message'][i]
    cleaned_email = clean_email(email_text)
    if cleaned_email is not None:
        cleaned_email = re.sub(r"[^a-zA-Z.!? ]+", "", cleaned_email.replace("!", ".").replace("?", "."))
        tokens = re.findall(r'\w+|[^\w\s]', cleaned_email.lower(), re.UNICODE)
        if '@' in tokens:
            print('@')
        full_corpus.append(['@','@']+tokens)

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 2349.79it/s]


In [13]:
word_completor = WordCompletor(full_corpus)

100%|██████████████████████████████████| 2241/2241 [00:00<00:00, 373749.86it/s]


In [14]:
n_gram_model = NGramLanguageModel(corpus=full_corpus, n=3)

100%|████████████████████████████████████████| 99/99 [00:00<00:00, 5500.69it/s]


In [15]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

In [ ]:
out = text_suggestion.suggest_text(['here', 'is', 'm'], n_words=4, n_texts=10)